In [53]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import random as rn
rn.seed(12345)
from numpy.random import seed
seed(42)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(42)

import yaml

import tensorflow_probability as tfp
import nipy as ni
import nibabel as nib
import numpy as np

from tensorflow.keras.models import load_model
from GPUtil import getFirstAvailable
from tensorflow.keras import backend as K
from hashlib import md5

import warnings
warnings.filterwarnings('ignore')

use only one trk/dwi pair for now, TODO: average over both instances
OR: implement matching fibers over the two instances
predicted fibers should be resampled only at predicted points (so that we don't smooth it out too much)
TODO: calculate separate prior agreement

In [57]:
def agreement(config):
    
    hasher = md5()
    for v in config.values():
        hasher.update(str(v).encode())
    
    subject_dir = os.path.join("subjects", config["subject"])
    
    save_dir = os.path.join(subject_dir, "agreement", hasher.hexdigest())
    if os.path.exists(save_dir):
        print("Agreement with this config has been calculated already:\n{}".format(save_dir))
        return
    
    sample_path_1 = os.path.join(subject_dir, "samples", config["sample_dirs"][0], "samples.npz")
    samples_1 = np.load(sample_path_1)
    
    def negative_log_likelihood(observed_y, predicted_distribution):
        return -K.mean(predicted_distribution.log_prob(observed_y))
    
    model_path_1 = os.path.join("models", config["model_dirs"][0], "model.h5")
    model_1 = load_model(
        model_path_1,
        custom_objects={"negative_log_likelihood": negative_log_likelihood,
                        "DistributionLambda": tfp.layers.DistributionLambda})
    
    model_path_2 = os.path.join("models", config["model_dirs"][1], "model.h5")
    model_2 = load_model(
        model_path_2,
        custom_objects={"negative_log_likelihood": negative_log_likelihood,
                        "DistributionLambda": tfp.layers.DistributionLambda})
    
    def c(kappa):
        return kappa / (4 * np.pi * np.sinh(kappa))
    
    batch_size = 2**15 # 32768
    n_samples = len(samples_1["inputs"])
    n_batches = np.ceil(n_samples / batch_size).astype(int)
    
    local_agreement = np.array([])
    for i in range(3):#n_batches):
    
        posterior_1 = model_1(samples_1["inputs"][i*batch_size:(i+1)*batch_size])
        posterior_2 = model_2(samples_1["inputs"][i*batch_size:(i+1)*batch_size])

        mu_1 = posterior_1.mean()
        mu_2 = posterior_2.mean()

        kappa_1 = posterior_1.concentration.numpy()
        kappa_2 = posterior_2.concentration.numpy()

        kappa_12 = np.linalg.norm(mu_1 + mu_2, axis=1)

        agreement = c(kappa_1) * c(kappa_2) / c(kappa_12)
        
        local_agreement = np.hstack([local_agreement, agreement])
        
        print("Finished {:4d}/{} batches.".format(i+1, n_batches), end="\r")
    
    os.makedirs(save_dir)
    
    config_path = os.path.join(save_dir, "config.yml")
    print("Saving {}".format(config_path))
    config["agreement_mean"] = str(local_agreement.mean())
    config["agreement_std"] = str(local_agreement.std())
    with open(config_path, "w") as file:
        yaml.dump(config, file, default_flow_style=False) 
    
    return local_agreement

In [41]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(getFirstAvailable(order="load", maxLoad=10**-6, maxMemory=10**-1)[0])

In [51]:
config = dict(
    subject = "992774",
    sample_dirs = ["fa7c02604b92de5f32cd3b61dbc2f8b7", "samples_2"],
    model_dirs = ["entrack_conditional/8d5593b08d4548286cc8564373e82e11",
                  "entrack_conditional/8d5593b08d4548286cc8564373e82e11"]
)

In [58]:
local_agreement = agreement(config)

Saving subjects/992774/agreement/a0f910610ec7b183514e22fad9bace7c/config.yml
